In [4]:
from __future__ import annotations
import numpy as np
import math
from dataclasses import dataclass
from typing import Literal, Tuple, Dict, List
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass, replace as dc_replace

### Q1.3

In [2]:
@dataclass
class params:
    S0: float      # initial asset price
    v0: float      # initial variance
    r: float       # risk‑free rate
    kappa: float   # mean‑reversion speed of v_t
    theta: float   # long‑run variance
    xi: float      # volatility of volatility ("vol‑of‑vol")
    rho: float     # correlation between dW_S and dW_v
    T: float       # option maturity in years
    K: float       # strike price
    N: int         # number of time steps
    M: int         # number of Monte Carlo paths

p = params(
    S0=100.0,
    v0=0.04,  # 20% vol²
    r=0.06,
    kappa=2.0,
    theta=0.04,
    xi=0.25,
    rho=-0.9,
    T=1.0,
    K=100.0,
    N=1000,
    M=100000
)


In [3]:
class AssetPath():
    def __init__(self, p: params):
        self.parameters = p
        self.Z1, self.Z2 = self.correlated_normals(p.rho, [p.M,p.N])

    def correlated_normals(self, rho: float, size: Tuple[int, int]) -> Tuple[np.ndarray, np.ndarray]:
        np.random.seed(42)
        z1 = np.random.normal(size=size)
        z2 = np.random.standard_normal(size=size)
        z2 = rho * z1 + np.sqrt(1.0 - rho**2) * z2
        return z1, z2

    def simulate_gbm_paths(self, scheme: str) -> np.ndarray:
        dt = self.parameters.T / self.parameters.N
        sqrt_dt = np.sqrt(dt)

        S = np.zeros((self.parameters.M, self.parameters.N + 1))
        S[:, 0] = self.parameters.S0

        dW_S = sqrt_dt * self.Z1
        for n in range(self.parameters.N):
            S_prev = S[:, n]

            if scheme == "euler":

                S_next = S_prev * np.exp(
                    (self.parameters.r - 0.5 * p.v0) * dt + np.sqrt(p.v0) * dW_S[:, n]
                )

            elif scheme == "milstein":

                S_next = S_prev * np.exp(
                    (self.parameters.r - 0.5 * p.v0) * dt + np.sqrt(p.v0) * dW_S[:, n] 
                )

            S[:, n + 1] = S_next

        return S

In [4]:
assetpath = AssetPath(p)

BS_euler_asset = assetpath.simulate_gbm_paths(scheme='euler')
BS_milstein_asset = assetpath.simulate_gbm_paths(scheme='milstein')

In [5]:
def MS_binary_price(S: np.ndarray, p: params) -> Tuple[float, float]:
    final_price = S[:,-1]
    payoff = np.exp(-p.r*p.T)*(final_price > p.K).astype(int)
    price = payoff.mean()
    stderr = payoff.std(ddof=1) / np.sqrt(p.M)
    return price, stderr, payoff

def analytical_binary_price(p: params) -> float:
    d = (np.log(p.S0/p.K) + (p.r - p.v0/2)*p.T)/np.abs(p.v0)*np.sqrt(p.T)
    return np.exp(-p.r*p.T)*norm.cdf(d)

In [9]:
euler_price, euler_stderr, euler_payoff = MS_binary_price(BS_euler_asset, p)
milstein_price, milstein_stderr, milstein_payoff = MS_binary_price(BS_milstein_asset, p)

bs_ref = analytical_binary_price(p)

In [10]:
print("\nSanity GBM: MC Euler = {:.4f}  Analytic = {:.4f}  |Δ| = {:.4e}".format(euler_price, bs_ref, abs(euler_price - bs_ref)))
print("\nSanity GBM: MC Milstein = {:.4f}  Analytic = {:.4f}  |Δ| = {:.4e}".format(milstein_price, bs_ref, abs(milstein_price - bs_ref)))


Sanity GBM: MC Euler = 0.5436  Analytic = 0.7923  |Δ| = 2.4875e-01

Sanity GBM: MC Milstein = 0.5436  Analytic = 0.7923  |Δ| = 2.4875e-01


### Q1.4

In [54]:
@dataclass
class FD_params:
    S0: float       # Initial asset price
    S_min: float   # Minimum asset price
    S_max: float   # Maximum asset price
    v0: float      # initial variance
    r: float       # risk‑free rate
    kappa: float   # mean‑reversion speed of v_t
    theta: float   # long‑run variance
    xi: float      # volatility of volatility ("vol‑of‑vol")
    rho: float     # correlation between dW_S and dW_v
    T: float       # option maturity in years
    K: float       # strike price
    N: int         # number of time steps
    M: int         # number of stock steps

p = FD_params(
    S0=100.0,
    S_min=90,
    S_max=110,
    v0=0.04,  # 20% vol²
    r=0.06,
    kappa=2.0,
    theta=0.04,
    xi=0.25,
    rho=-0.9,
    T=1.0,
    K=100.0,
    N=1000,
    M=1000
)


In [67]:
def implicit(p:FD_params) -> float:
    q = 2*p.r/p.v0
    a = 0.5*(q-1)
    b = 0.25*(q + 1)**2
    x = np.linspace(np.log(p.S_min/p.K), np.log(p.S_max/p.K), p.M)
    t  = np.linspace(0, 0.5*p.v0*p.T, p.N)

    dt = (0.5*p.v0*p.T)/p.N
    dx = x[1] - x[0]
    tau = dt
    lamda = dt/dx**2

    diag = np.ones(p.M) * (1+2*lamda)
    off_diag = np.ones(p.M-1) * -lamda
    A = np.diag(diag) + np.diag(off_diag, k=1) + np.diag(off_diag, k=-1)
    A_inv = np.linalg.inv(A)

    y = np.zeros([p.N, p.M], dtype=np.float128)
    y[0,:] = np.where(np.exp(0.5*(q+1)*x) > np.exp(0.5*(q-1)*x), 1, 0)
    
    for i in range(1,p.N):
        y[i,:] = np.matmul(A_inv, y[i-1,:])
        y[i,-1] += lamda*2*dx*np.exp((a+1)*np.log(p.S_max/p.K) + b*tau)
        tau+=dt
    # price = (y[p.N-1,:]*p.K)/(np.exp(a*x + b*p.v0*p.T*0.2))
    price = np.interp(np.log(p.S0/p.K), x, y[-1,:].astype(np.float64))

    return price*p.K*np.exp(a*np.log(p.S0/p.K)+b*p.v0*0.5*p.T)


In [68]:
price = implicit(p)
print(price)

0.5174891507801304


In [ ]:
def crank_nicolson(p:FD_params):
    q = 2*p.r/p.v0
    a = 0.5*(q-1)
    b = 0.25*(q + 1)**2
    x = np.linspace(np.log(p.S_min/p.K), np.log(p.S_max/p.K), p.M)

    dt = (0.5*p.v0*p.T)/p.N
    dx = x[1] - x[0]
    tau = dt
    lamda = dt/dx**2

    diag1 = np.ones(p.M) * (1+lamda)
    diag2 = np.ones(p.M) * (1-lamda)
    off_diag = np.ones(p.M-1) * lamda/2
    A1 = np.diag(diag1) + np.diag(-off_diag, k=1) + np.diag(-off_diag, k=-1)
    A2 = np.diag(diag2) + np.diag(off_diag, k=1) + np.diag(off_diag, k=-1)
    A1_inv = np.linalg.inv(A1)
    A2_inv = np.linalg.inv(A2)

    y = np.zeros([p.N, p.M], dtype=np.float128)
    y[0,:] = np.where(np.exp(0.5*(q+1)*x) > np.exp(0.5*(q-1)*x), 1, 0)
    
    for i in range(1,p.N):
        tmp = np.matmul(A2_inv, y[i-1,:]) + lamda*2*dx*np.exp((a+1)*np.log(p.S_max/p.K) + b*tau) - lamda*2*dx*np.exp((a+1)*np.log(p.S_max/p.K) + b*(tau-dt))
        y[i,:] = np.matmul(A1_inv, tmp)
        tau+=dt
    price = np.interp(np.log(p.S0/p.K), x, y[-1,:].astype(np.float64))

    return price*p.K*np.exp(a*np.log(p.S0/p.K)+b*p.v0*0.5*p.T)


In [ ]:
price_cn = crank_nicolson(p)
print(price_cn)